In [1]:
!pip install simpy


In [6]:
#CELL 1 — Imports + Config class
import simpy
import random
import statistics

class Config:
    def __init__(self,
                 interarrival_mean=25.0,
                 prep_mean=40.0,
                 surg_mean=20.0,
                 rec_mean=40.0,
                 P=3,
                 R=3,
                 sim_time=20000,
                 monitor_interval=5.0,
                 seed=42):
        
        self.interarrival_mean = interarrival_mean
        self.prep_mean = prep_mean
        self.surg_mean = surg_mean
        self.rec_mean = rec_mean
        
        self.P = P                 # number of preparation rooms
        self.R = R                 # number of recovery beds
        
        self.sim_time = sim_time  # simulation horizon
        self.monitor_interval = monitor_interval
        
        self.seed = seed


In [4]:
#CELL 2 — Patient class
class Patient:
    def __init__(self, pid, arrival_time, cfg: Config):
        self.id = pid
        self.arrival_time = arrival_time
        
        # Draw personal service times (exponential distributions)
        self.prep_time = random.expovariate(1.0 / cfg.prep_mean)
        self.surg_time = random.expovariate(1.0 / cfg.surg_mean)
        self.rec_time = random.expovariate(1.0 / cfg.rec_mean)

In [5]:
#CELL 3 — Metrics container
def init_metrics():
    return {
        'n_completed': 0,
        'throughput_times': [],

        # OR busy tracking
        'or_busy_time': 0.0,
        'or_last_change': 0.0,
        'or_is_busy': False,

        # Monitoring data
        'queue_length_samples': [],
        'queue_length_sample_times': [],
        'or_busy_samples': [],
        'or_busy_sample_times': []
    }

In [7]:
#CELL 4 — Helper for OR busy timing
def set_or_busy(metrics, now, busy_flag: bool):
    """
    Keeps track of how long the operating theatre stays busy.
    """
    last_change = metrics['or_last_change']

    # If OR was busy until now, accumulate busy time
    if metrics['or_is_busy']:
        metrics['or_busy_time'] += (now - last_change)

    metrics['or_is_busy'] = busy_flag
    metrics['or_last_change'] = now

In [22]:
#CELL 5 — Patient Flow (core process)
def patient_flow(env, patient: Patient,
                 prep, operating_theatre, recovery,
                 metrics, cfg: Config):
    """
    Full lifecycle of one patient:
    Arrival -> Preparation -> Surgery -> (wait for Recovery bed) -> Recovery -> Exit
    """
    arrival_time = patient.arrival_time

    # ------------------- PREPARATION -------------------
    with prep.request() as req_prep:
        # Wait until a prep unit is free (entrance queue)
        yield req_prep

        # Perform preparation
        yield env.timeout(patient.prep_time)

        # ------------------- SURGERY + WAIT FOR RECOVERY BED -------------------
        with operating_theatre.request() as req_or:
            # Wait until OR is free
            yield req_or

            # OR becomes busy
            set_or_busy(metrics, env.now, True)

            # Surgery itself
            yield env.timeout(patient.surg_time)

            # After surgery, patient needs a recovery bed.
            # While we are waiting for a recovery bed, OR is still blocked.
            req_rec = recovery.request()
            yield req_rec   # wait until some recovery bed is free

            # Now a recovery bed is obtained → patient moves to Recovery,
            # OR becomes idle again.
            set_or_busy(metrics, env.now, False)

        # <-- Exiting the 'with operating_theatre...' block RELEASES the OR resource here.

        # ------------------- RECOVERY -------------------
        # Patient is now in a recovery bed (we hold req_rec).
        yield env.timeout(patient.rec_time)

        # Recovery finished → free the recovery bed.
        recovery.release(req_rec)

    # ------------------- EXIT -------------------
    metrics['n_completed'] += 1
    T_i = env.now - arrival_time
    metrics['throughput_times'].append(T_i)


In [10]:
#CELL 6 — Patient Generator (Arrival Process)
def patient_generator(env, prep, operating_theatre, recovery,
                      metrics, cfg: Config):
    """
    Generates patients following an exponential interarrival distribution.
    Each patient is assigned personal service times and enters patient_flow().
    """
    pid = 0
    while True:
        # Create new patient
        arrival_time = env.now
        patient = Patient(pid, arrival_time, cfg)
        pid += 1

        # Start patient's lifecycle as a SimPy process
        env.process(
            patient_flow(env, patient,
                         prep, operating_theatre, recovery,
                         metrics, cfg)
        )

        # Sample next interarrival time
        interarrival = random.expovariate(1.0 / cfg.interarrival_mean)
        yield env.timeout(interarrival)


In [12]:
#CELL 7 — Monitoring Process
def monitor(env, prep, operating_theatre, metrics, cfg: Config):
    """
    Periodically collects:
    - Entrance queue length (patients waiting for prep)
    - OR busy state
    
    This is for time-averaged statistics.
    """
    while True:
        now = env.now

        # Queue length = number of requests waiting for the prep resource
        entrance_q_len = len(prep.queue)

        metrics['queue_length_samples'].append(entrance_q_len)
        metrics['queue_length_sample_times'].append(now)

        metrics['or_busy_samples'].append(1 if metrics['or_is_busy'] else 0)
        metrics['or_busy_sample_times'].append(now)

        yield env.timeout(cfg.monitor_interval)


In [24]:
#CELL 8 — Simulation Runner (Core Engine)
def run_simulation(cfg: Config):
    """
    Sets up and runs the full hospital simulation.
    Returns:
        - results (dict of summary statistics)
        - metrics (raw collected data)
    """
    random.seed(cfg.seed)
    env = simpy.Environment()

    # ------------------- RESOURCES -------------------
    prep = simpy.Resource(env, capacity=cfg.P)                # preparation units
    operating_theatre = simpy.Resource(env, capacity=1)       # single OR
    recovery = simpy.Resource(env, capacity=cfg.R)            # recovery beds

    # ------------------- METRICS -------------------
    metrics = init_metrics()

    # ------------------- PROCESSES -------------------
    # Patient arrival generator
    env.process(patient_generator(env, prep, operating_theatre, recovery,
                                  metrics, cfg))

    # Monitoring process
    env.process(monitor(env, prep, operating_theatre, metrics, cfg))

    # ------------------- RUN SIMULATION -------------------
    env.run(until=cfg.sim_time)

    # Close last interval of OR busy time if OR still busy
    if metrics['or_is_busy']:
        metrics['or_busy_time'] += (env.now - metrics['or_last_change'])

    # ------------------- RETURN RESULTS -------------------
    results = summarize_results(env, metrics, cfg)
    return results, metrics


In [25]:
#CELL 9 — Result Summary Function
def summarize_results(env, metrics, cfg: Config):
    T = env.now   # total simulated time

    # ------------------- Throughput -------------------
    if metrics['throughput_times']:
        avg_throughput = statistics.mean(metrics['throughput_times'])
    else:
        avg_throughput = float('nan')

    # ------------------- Average entrance queue length -------------------
    if metrics['queue_length_samples']:
        avg_queue = statistics.mean(metrics['queue_length_samples'])
    else:
        avg_queue = float('nan')

    # ------------------- OR utilization -------------------
    # Exact utilization using accumulated busy time
    if T > 0:
        or_util_exact = metrics['or_busy_time'] / T
    else:
        or_util_exact = float('nan')

    # Approximation from monitoring samples
    if metrics['or_busy_samples']:
        or_util_mon = statistics.mean(metrics['or_busy_samples'])
    else:
        or_util_mon = float('nan')

    return {
        'simulation_time': T,
        'n_completed': metrics['n_completed'],
        'avg_throughput_time': avg_throughput,
        'avg_entrance_queue_length': avg_queue,
        'or_utilization_exact': or_util_exact,
        'or_utilization_monitored': or_util_mon
    }


In [27]:
#CELL 10 — Test Run the Simulation
# Create a configuration with the assignment parameters
cfg = Config(
    interarrival_mean=25.0,
    prep_mean=40.0,
    surg_mean=20.0,
    rec_mean=40.0,
    P=3,
    R=3,
    sim_time=20000,
    monitor_interval=5.0,
    seed=123
)

# Run the simulation
results, metrics = run_simulation(cfg)

# Print results
print("=== Simulation Results ===")
for k, v in results.items():
    print(f"{k}: {v}")


=== Simulation Results ===
simulation_time: 20000
n_completed: 555
avg_throughput_time: 3387.577987176837
avg_entrance_queue_length: 138.987
or_utilization_exact: 0.5540671852934531
or_utilization_monitored: 0.55325
